<a href="https://colab.research.google.com/github/rufous86/studies/blob/main/recommender_systems/RecSys_on_practice_kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.kaggle.com/competitions/recsys-in-practice/data?select=train_joke_df.csv

In [1]:
! pip install -q kaggle
from google.colab import drive
drive.mount('/content/drive')
! mkdir ~/.kaggle
! cp '/content/drive/MyDrive/Colab Notebooks/kaggle.json' ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle competitions download -c recsys-in-practice
! unzip recsys-in-practice.zip -d .

Mounted at /content/drive
  0% 0.00/11.2M [00:00<?, ?B/s]
100% 11.2M/11.2M [00:00<00:00, 129MB/s]
Archive:  recsys-in-practice.zip
  inflating: ./sample_submission.csv  
  inflating: ./test_joke_df_nofactrating.csv  
  inflating: ./train_joke_df.csv     


In [2]:
! pip install scikit-surprise -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 16.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [9]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

import scipy.sparse as sp
from scipy.sparse import csr_matrix

In [4]:
df_train = pd.read_csv('train_joke_df.csv')
df_train

,UID,JID,Rating
0,18029,6,-1.26
1,3298,64,-4.17
2,3366,58,0.92
3,12735,92,3.69
4,11365,38,-6.60
...,...,...,...
1448359,22604,26,2.82
1448360,22255,36,-1.94
1448361,21056,40,-9.56
1448362,12328,97,0.87


In [5]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1448364 entries, 0 to 1448363
Data columns (total 3 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   UID     1448364 non-null  int64  
 1   JID     1448364 non-null  int64  
 2   Rating  1448364 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 33.2 MB


In [6]:
df_train.describe()

,UID,JID,Rating
count,1.448364e+06,1.448364e+06,1.448364e+06
mean,1.252160e+04,4.347179e+01,8.817123e-01
std,7.198297e+03,2.619797e+01,5.234737e+00
min,1.000000e+00,1.000000e+00,-9.950000e+00
25%,6.283000e+03,2.100000e+01,-2.960000e+00
50%,1.255400e+04,4.200000e+01,1.500000e+00
75%,1.875000e+04,6.300000e+01,5.100000e+00
max,2.498300e+04,1.000000e+02,1.000000e+01


In [54]:
df_train['UID'].nunique(), df_train['JID'].nunique(), df_train['Rating'].nunique()

(24983, 100, 412)

In [53]:
# df_coo = df_train.pivot_table(values='Rating', index='UID', columns='JID')
# df_coo = df_coo.fillna(-1e-3)
# df_coo

In [56]:
from surprise import accuracy, Dataset, SVD
from surprise import Dataset, NormalPredictor, Reader
from surprise.model_selection import train_test_split

reader = Reader(rating_scale=(-10, 10))
data = Dataset.load_from_df(df_train[["UID", "JID", "Rating"]], reader)

trainset, testset = train_test_split(data, test_size=0.25)


algo = SVD()

# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.test(testset)

# Then compute RMSE
accuracy.rmse(predictions)

RMSE: 4.1886


4.188613003277133

In [63]:
uid = str(11529)  # raw user id (as in the ratings file). They are **strings**!
iid = str(1)  # raw item id (as in the ratings file). They are **strings**!

# get a prediction for specific users and items.
pred = algo.predict(uid, iid, verbose=True)

user: 11529      item: 1          r_ui = None   est = 0.88   {'was_impossible': False}


In [58]:
df_test = pd.read_csv('/content/test_joke_df_nofactrating.csv')
df_test.head(3)

,InteractionID,UID,JID
0,0,11228,39
1,1,21724,85
2,2,16782,56


In [ ]:
preds = []
for row in df_test.iterrows():
    uid = str(row[1]['UID'])
    iid = str(row[1]['JID'])
    pred = algo.predict(uid, iid, verbose=True)
    preds.append(pred[3])